## 1️⃣ Verificar GPU e CUDA

In [ ]:
import torch
import subprocess

print("="*70)
print("VERIFICAÇÃO DE GPU")
print("="*70)

# Check CUDA
print(f"\n✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA disponível: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA versão: {torch.version.cuda}")
    print(f"✓ cuDNN: {torch.backends.cudnn.version()}")
    
    props = torch.cuda.get_device_properties(0)
    print(f"✓ Memória GPU: {props.total_memory / 1e9:.1f} GB")
    
    # nvidia-smi
    print("\n--- nvidia-smi ---")
    subprocess.run(['nvidia-smi'])
else:
    print("\n⚠️  AVISO: GPU não detectada!")
    print("   Vá em Runtime > Change runtime type e selecione GPU")

print("\n" + "="*70)

In [ ]:
import os
import subprocess

# Check if already cloned
REPO_PATH = '/content/Xadrez_AI_Final'

if os.path.exists(REPO_PATH):
    print(f"✓ Repositório já existe em {REPO_PATH}")
    os.chdir(REPO_PATH)
else:
    print(f"Clonando repositório...")
    os.chdir('/content')
    subprocess.run([
        'git', 'clone', 
        'https://github.com/devolopbomfim/Xadrez_AI_Final.git'
    ])
    os.chdir(REPO_PATH)

print(f"\n✓ Diretório de trabalho: {os.getcwd()}")
print(f"\nArquivos principais:")
for item in ['training/', 'models/', 'core/', 'engine/']:
    if os.path.exists(item):
        print(f"  ✓ {item}")

## 4️⃣ Configurar Paths e Verificar Ambiente

In [ ]:
import sys
import os

# Add to path
sys.path.insert(0, '/content/Xadrez_AI_Final')

# Criar diretórios se necessário
os.makedirs('models/Agente02/checkpoints', exist_ok=True)

print("✓ Path configurado")
print(f"✓ Diretório de trabalho: {os.getcwd()}")

# Test imports
try:
    from training.run_heavy import run_heavy
    print("✓ Import training.run_heavy OK")
except Exception as e:
    print(f"✗ Erro ao importar: {e}")

## 5️⃣ Configurar Variáveis de Treinamento

In [ ]:
# Configuração do Agente02 (intermediário)
CONFIG = {
    'agent_name': 'Agente02',
    'num_selfplay': 256,           # Quantidade de jogos
    'selfplay_sims': 512,          # Simulações MCTS por movimento
    'trainer_iters': 2000,         # Iterações de treinamento
    'batch_size': 256,             # Tamanho do batch (otimizado para GPU)
    'model_channels': 160,         # Canais da rede neural
    'model_blocks': 16,            # Blocos residuais
    'use_reward_shaping': True,    # Usar recompensas incrementais
    'use_gpu': True,               # Usar GPU
}

print("="*70)
print("CONFIGURAÇÃO DE TREINAMENTO")
print("="*70)
print(f"\nAgente: {CONFIG['agent_name']}")
print(f"\nParâmetros de Self-Play:")
print(f"  • Games: {CONFIG['num_selfplay']}")
print(f"  • MCTS simulations/move: {CONFIG['selfplay_sims']}")
print(f"\nParâmetros de Treinamento:")
print(f"  • Training iterations: {CONFIG['trainer_iters']}")
print(f"  • Batch size: {CONFIG['batch_size']}")
print(f"\nArquitetura:")
print(f"  • Channels: {CONFIG['model_channels']}")
print(f"  • Blocks: {CONFIG['model_blocks']}")
print(f"\nOpções:")
print(f"  • Reward shaping: {CONFIG['use_reward_shaping']}")
print(f"  • GPU: {CONFIG['use_gpu']}")
print(f"\nEstimativa de tempo: ~12-18 horas com GPU T4")
print("="*70)

## 6️⃣ Executar Treinamento (PRINCIPAL)

In [ ]:
import torch
import os
from datetime import datetime

# Import training function
from training.run_heavy import run_heavy

print("="*80)
print(f"INICIANDO TREINAMENTO - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)
print()

# Verificar GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo: {device.upper()}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print()

# Executar treinamento
try:
    result = run_heavy(
        num_selfplay=CONFIG['num_selfplay'],
        selfplay_sims=CONFIG['selfplay_sims'],
        trainer_iters=CONFIG['trainer_iters'],
        batch_size=CONFIG['batch_size'],
        model_kwargs={
            'channels': CONFIG['model_channels'],
            'blocks': CONFIG['model_blocks'],
            'action_size': 20480
        },
        agent_dir='models/Agente02',
        preferred_device=device,
        use_reward_shaping=CONFIG['use_reward_shaping']
    )
    
    print("\n" + "="*80)
    if result == 0:
        print("✓✓✓ TREINAMENTO CONCLUÍDO COM SUCESSO!")
    else:
        print(f"✗✗✗ Treinamento falhou com código: {result}")
    print("="*80)
    
except Exception as e:
    print(f"\n✗ Erro durante treinamento: {e}")
    import traceback
    traceback.print_exc()

## 7️⃣ Verificar Resultados

In [ ]:
import os

print("="*70)
print("RESULTADOS DO TREINAMENTO")
print("="*70)

checkpoint_dir = 'models/Agente02/checkpoints'

if os.path.exists(checkpoint_dir):
    files = os.listdir(checkpoint_dir)
    print(f"\nArquivos em {checkpoint_dir}:")
    
    for fname in sorted(files):
        fpath = os.path.join(checkpoint_dir, fname)
        size_mb = os.path.getsize(fpath) / (1024**2)
        print(f"  ✓ {fname}: {size_mb:.1f} MB")
    
    # Informações importantes
    print("\n✓ Checkpoints salvos com sucesso!")
    print("\nPróximos passos:")
    print("  1. Fazer download dos modelos")
    print("  2. Copiar para ambiente local")
    print("  3. Testar performance")
else:
    print(f"\n✗ Diretório {checkpoint_dir} não encontrado")

## 8️⃣ Download dos Modelos (Para Local)

In [ ]:
import os
from google.colab import files

print("Preparando arquivos para download...\n")

checkpoint_dir = 'models/Agente02/checkpoints'

if os.path.exists(checkpoint_dir):
    # List files
    files_to_download = [
        os.path.join(checkpoint_dir, f) 
        for f in os.listdir(checkpoint_dir)
    ]
    
    print(f"Arquivos disponíveis para download:")
    for f in files_to_download:
        size_mb = os.path.getsize(f) / (1024**2)
        print(f"  • {f} ({size_mb:.1f} MB)")
    
    print("\n⏳ Para fazer download, execute:")
    print("\nfrom google.colab import files")
    print("files.download('models/Agente02/checkpoints/latest.pt')")
    print("files.download('models/Agente02/checkpoints/replay.pt')")
else:
    print(f"✗ Diretório {checkpoint_dir} não existe")

## 9️⃣ Limpeza de Memória

In [ ]:
import torch
import gc

print("Limpando memória...")

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ GPU cache limpo")

# Garbage collection
gc.collect()
print("✓ Garbage collection executado")

print("\n✓ Limpeza concluída!")

## ℹ️ Informações Importantes

### 📊 Configuração do Agente02:
- **Games**: 256 (4x mais que Agente01)
- **MCTS Simulations**: 512 por movimento (4x mais)
- **Training Iterations**: 2000
- **Batch Size**: 256 (otimizado para GPU)
- **Modelo**: 160 channels, 16 blocos residuais
- **Reward Shaping**: ✓ Ativo (30% step + 70% final)

### ⏱️ Tempo de Execução:
- **GPU T4**: ~12-18 horas
- **GPU P100**: ~6-8 horas
- **GPU V100**: ~3-4 horas

### 💾 Arquivos Gerados:
- `latest.pt` (~24 MB) - Modelo treinado
- `replay.pt` (~500-600 MB) - Buffer de replay com games

### 🔗 Links Úteis:
- [Repositório](https://github.com/devolopbomfim/Xadrez_AI_Final)
- [PyTorch Docs](https://pytorch.org/docs/stable/index.html)
- [Google Colab Docs](https://colab.research.google.com/)

### ❓ Troubleshooting:
- **GPU não detectada**: Vá em `Runtime > Change runtime type > GPU`
- **Falta de memória**: Reduza `batch_size` de 256 para 128
- **Timeout**: O Colab pode desconectar, mas o treinamento continua (esconda o notebook)
- **Conexão perdida**: Reconecte e verifique os checkpoints salvos